## GESTDB HACKATON GRUPO 1: OPENALEX

### PARTICIPANTES:
* Jaime Vaquero Rabahieh. Correo: jaime.vaquero@alumnos.upm.es
* Zakaria Lasry Sahraoui. Correo: z.lsahraoui@alumnos.upm.es
* Damian Sanchez Maqueda. Correo: damian.sanchez@alumnos.upm.es
* Radu-Andrei Bourceanu. Correo: r.bourceanu@alumnos.upm.es

## PLANTEAMIENTO Y ACOTACIÓN DE OBJETVIOS INICIALES
---

El propósito inicial busca responder a qué instituciones y países son los "hubs" de conocimiento actuales. Para ello, se analizará cuáles colaboran más con otros centros y publican más sobre una temática concreta.

Las problemáticas que hallamos con esta elección eran la siguientes:

* No utilizabamos ninguna fuente de datos no estructurados: Optamos por dirigir el proyecto a un enfoque diferente que parte del interés de comprender que tecnologías de programación son las mas usadas, en artículos o proyectos de investigación relacionados con la computación, y como varía su uso y distribución en función de cada subcampo. En concreto en este proyecto nos hemos centrado en la en área de la Inteligencia Artifical, por ser la que tiene mayor relación con el mater que actualmente nos encontramos cursando.


A continuación se muestra la imagen de la estructura completa de la base de datos del proyecto.

![Diagrama Entidad-Relación del sistema](./images/relaciones%20db.png)


* La cantidad de PDF a gestionar era demasiado amplia: 
OpenAlex cuenta con un total de **200 Millones de archivos**, lo cual implicaría una cantidad alarmante de tiempo para extraer y procesar toda la información necesaria para cumplir el objetivo. Por lo tanto, decidimos utilizar el campo _Temática_ para filtrar el número de archivos a extraer. Para poder obtener nuestro objetivo (extraer tecnologías por tema), decidimos filtrar una sucesión de temáticas padre-hijo relacionadas con nuestra meta: ***Computer_Science -> Artificial Inteligence -> Temáticas sin hijo***. También fue necesario filtrar por archivos con Pdf y que puedan ser compatibles con el LLM (OpenAI). Como consecuencia, el número de obras a extraer se redució a 6000. La siguiente imagen es una representación de dicho filtro dentro de OpenAlex:

![Filtro temáticas dentro de OpenAlex](./images/filtro_openalex.jpeg)

En base a esta decisión además decidimos acotar el dominio de entidades para abordar lo suficiente para hacer una demostración en el Hackathon:

![Arquitectura reducida del sistema](./images/esquema_redux.jpeg)

## Especificaciones técnicas por fase

### Datos estructurados
Mediante **OpenAlex**, una base de datos abierta de investigaciones científicas a nivel global, similar a Google Scholar, pero con datos accesibles mediante API, podemos extraer articulos academicos, incluyendo metadatos como su dirección fuente, abstract, numero de citas, autores e instituciones asociadas al autor en el momento de la publicación.


Por ejemplo:

> Andrea Cimmino publicó un artículo durante su estancia en la UPM, universidad española y europea, sobre NLP y siendo este desarrollado principalmente en Python


Usando la biblioteca `requests` para hacer llamadas a la **API de OpenAlex**, extrayendo así el contenido descrito y almacenarlo en una base de datos de **PostgreSQL**.



### Datos no estructurados

Sobre esta base de datos estructurada, aplicamos un proceso de análisis de texto mediante la API de **OpenAI** para identificar para cada obra cual ha sido la tecnología utilizada. Esto lo hemos realizado pasandole a `gpt-5-nano` la url de cada obra para que así pueda identificar la tecnología usada sin necesidad de guardar una copia de cada obra en la base de datos. Todas la tecnologías conocidas por nuestra base de datos se almacenan como una entidad, y se relacionan directamente con las obras en la que el modelo haya detectado su uso la misma. Esta estructura permite explorar facilmente tendencias tecnológicas y la difusión de herramientas de programación dentro del ámbito científico.

### Datos enlazados



Una vez terminado el procesamiento de los documentos y habiendo extraído las tecnologías de cada obra estás insertadas también a la BBDD **PostgreSQL**, para construir el grafo de nuestro sistema, hemos utilizado **GraphDB** y así poder realizar las consultas necesarias del obetivo principal. Para facilitar este proceso hemos utilizado **RDF Turtle**.

Con el propósito de enriquecer semánticamente los datos extraídos y facilitar su interoperabilidad con otras fuentes abiertas, hemos seleccionado diversos vocabularios, ontologías y grafos de conocimiento ampliamente utilizados en el ecosistema de la web semántica. Cada uno de ellos contribuye a describir un tipo de entidad diferente dentro del modelo propuesto (obras, autores, instituciones o lenguajes de programación). El modelo combina información estructurada con elementos semánticos, utilizando Schema.org para describir los lenguajes y tecnologías detectadas, y SKOS para organizar jerárquicamente las entidades del dominio (obras, temas, etc.), lo que facilita la interoperabilidad y la consulta avanzada de los datos.


## TRABAJO REALIZADO EN EL HACKATHON

### CREACIÓN BASE DE DATOS POSTGRESQL
---

Y ya con el sistema filtrado, pudimos empezar a programarlo. Lo primero fue crear la base de datos en PostgreSQL, para lo cual declaramos los parametros de la BD (**host**, **port**, **database**, **user** y **password**) para lrealizar la conexión y obtener el cursosr. Posteriormente, escribimos en _tablas SQL_ las entidades _Obra_, _Tecnologia_ y _Tematica_ y las relaciones entre temáticas padre e hijo (*Temática_contenida*) y entre obras y tecnologías (*obra_tecnología*) Cabe destacar que la relación entre _Obra_ y _Tematica_ se encuentra representada por la Foreign Key *tecnologia_id*. Además, se crean índices externos para ayudar a la hora de organizar los datos en las tablas.

Por último, el sistema se conecta con el servidor de PostgreSQL para crear las tablas, por lo que es necesario **lanzar antes la imagen de docker de PostgreSQL para que funcione**. Si todo sale bien, al final se hace un commit con los cmabios y se cierran el ursor y la conexión.

In [ ]:
import psycopg2
from psycopg2 import sql

DB_PARAMS = {
    "host": "localhost",
    "port": 5432,
    "database": "demoDB",
    "user": "userPSQL",
    "password": "passPSQL"
}

sql_script = """CREATE TABLE IF NOT EXISTS tematica (
    id INTEGER,
    nombre_campo TEXT NOT NULL,
        PRIMARY KEY (id)
);

CREATE TABLE IF NOT EXISTS tecnologia (
    id INTEGER,
    nombre TEXT NOT NULL,
    tipo TEXT,
    version TEXT,
    PRIMARY KEY (id)
);

CREATE TABLE IF NOT EXISTS obra (
    id INTEGER,
    doi TEXT UNIQUE,  -- NEW
    direccion_fuente TEXT NOT NULL,
    titulo TEXT NOT NULL,
    abstract TEXT,
    fecha_publicacion TEXT,
    idioma TEXT,
    num_citas INTEGER DEFAULT 0,
    fwci REAL,
    tematica_id INTEGER,
    PRIMARY KEY (id),
    FOREIGN KEY (tematica_id)
        REFERENCES tematica(id)
        ON UPDATE CASCADE
        ON DELETE SET NULL
);

CREATE TABLE IF NOT EXISTS obra_tecnologia (
    id INTEGER,
    obra_id INTEGER NOT NULL,
    tecnologia_id INTEGER NOT NULL,
    PRIMARY KEY (id),
    FOREIGN KEY (obra_id)
        REFERENCES obra(id)
        ON UPDATE CASCADE
        ON DELETE CASCADE,
    FOREIGN KEY (tecnologia_id)
        REFERENCES tecnologia(id)
        ON UPDATE CASCADE
        ON DELETE RESTRICT
);

CREATE TABLE IF NOT EXISTS tematica_contenida (
    id INTEGER,
    tematica_padre_id INTEGER NOT NULL,
    tematica_hijo_id INTEGER NOT NULL,
    PRIMARY KEY (id),
    FOREIGN KEY (tematica_padre_id)
        REFERENCES tematica(id)
        ON UPDATE CASCADE
        ON DELETE CASCADE,
    FOREIGN KEY (tematica_hijo_id)
        REFERENCES tematica(id)
        ON UPDATE CASCADE
        ON DELETE CASCADE,
    CHECK (tematica_padre_id <> tematica_hijo_id)
);

CREATE INDEX IF NOT EXISTS idx_obra_tematica ON obra(tematica_id);
CREATE INDEX IF NOT EXISTS idx_obratec_tecnologia ON obra_tecnologia(tecnologia_id);
CREATE INDEX IF NOT EXISTS idx_tematica_hijo ON tematica_contenida(tematica_hijo_id);
"""

def main():
    # Connect to default database to check/create demoDB
    connection = psycopg2.connect(
        host=DB_PARAMS['host'],
        port=DB_PARAMS['port'],
        database="demoDB",
        user=DB_PARAMS['user'],
        password=DB_PARAMS['password']
    )
    connection.autocommit = True
    cursor = connection.cursor()

    cursor.execute("SELECT 1 FROM pg_database WHERE datname = %s;", (DB_PARAMS["database"],))
    exists = cursor.fetchone()

    if not exists:
        cursor.execute(sql.SQL(f"CREATE DATABASE {DB_PARAMS['database']};"))
        print(f"✅ Database '{DB_PARAMS['database']}' created.")
    else:
        print(f"ℹ️ Database '{DB_PARAMS['database']}' already exists.")

    cursor.close()
    connection.close()

    # Connect to demoDB to create tables
    connection = psycopg2.connect(**DB_PARAMS)
    cursor = connection.cursor()
    cursor.execute(sql_script)
    connection.commit()
    cursor.close()
    connection.close()
    print("✅ Tables created or verified successfully.")

if __name__ == "__main__":
    main()

ℹ️ Database 'demoDB' already exists.
✅ Tables created or verified successfully.


### OBTENCIÓN DATOS ESTRUCTURADOS (REQUEST)
---

Aunque nuestra idea siempre fue el uso de la librería _pyalex_ para extraer los datos estructurados, al final decidimos cambiarla por _requests_ ya que _pyalex_ daba problemas al extraer datos con símbolos no alfanuméricos. Eso sí, se mantuvo la idea de crear los csvs de cada entidad (_Obra_, _Tecnologia_ y _Tematica_) y también para las relaciones con tabla en PostGreSQL (*obra_tecnologia* y *tematica_contenida*). Se ha creado una carpeta **Cache** para almacenar los csvs.

La idea de este algoritmo es obtener los datos pedidos en cada entidad y relación, después limpiarlos y por último organizarlos y almacenarlos en cada csv correspondiente. El propio algoritmo envía a OpenAlex el filtro de búsqueda antes indicado y obtiene la información de todos los ficheros resultantes. Con cada fichero, obtiene solamente los datos que se relacionen con los distintos atributos de cada entidad y relación creada y estos son limpiados y almacenados en el diccionario correspondiente al csv donde deben aparecer (puede haber datos como los _ids_ que pueden aparecer en más de un csv). Por último, se crean los csvs si no existen y se guarda cada diccionario donde le correpsonde.

Cabe destacar que la extarcción es por **páginas de 200 entradas**. Por tanto, si llega a una página número J donde _J x 200 > número de papers detectados_, pues se termina la extracción. 

In [ ]:
import requests
import csv
import time
import os

BASE_URL = "https://api.openalex.org/works"
PER_PAGE = 200
KEYWORDS = [
    "python","c-programming-language","javascript","java","java-programming-language",
    "sql","dart","swift","cobol","fortran","matlab","prolog","lisp","haskell","rust","perl",
    "scala","html","html5"
]
SUBFIELD_ID = "subfields/1702"
LANGUAGE = "languages/en"
BASE_DIR = os.path.dirname(os.path.dirname(os.path.abspath(__file__)))  # go up one level from src/
CACHE_DIR = os.path.join(BASE_DIR, "cache")

CSV_OBRA = os.path.join(CACHE_DIR, "obra.csv")
CSV_TEMATICA = os.path.join(CACHE_DIR, "tematica.csv")
CSV_TEMATICA_CONTENIDA = os.path.join(CACHE_DIR, "tematica_contenida.csv")

os.makedirs(CACHE_DIR, exist_ok=True)

BASE_TOPICS = ["Physical Sciences", "Computer Science", "Artificial Intelligence"]

def reconstruct_abstract(abstract_inverted_index):
    if not abstract_inverted_index or not isinstance(abstract_inverted_index, dict):
        return ""
    position_map = {}
    for word, positions in abstract_inverted_index.items():
        for pos in positions:
            position_map[pos] = word
    return " ".join(position_map[pos] for pos in sorted(position_map.keys()))

def fetch_page(url, params, max_retries=5, delay_base=2):
    retries = 0
    while retries < max_retries:
        try:
            response = requests.get(url, params=params, timeout=30)
            if response.status_code == 200:
                return response.json()
            print(f"⚠️ Warning: Bad response {response.status_code}, retrying...")
        except Exception as e:
            print(f"⚠️ Warning: Exception during request: {e}")
        retries += 1
        time.sleep(delay_base ** retries)
    print(f"❌ Error: Failed to fetch page after {max_retries} retries.")
    return None

def initialize_csv_files():
    os.makedirs(os.path.dirname(CSV_OBRA), exist_ok=True)
    with open(CSV_OBRA, "w", newline='', encoding='utf-8') as f:
        writer = csv.writer(f)
        writer.writerow([
            "id","direccion_fuente","titulo","abstract","fecha_publicacion",
            "idioma","num_citas","fwci","tematica_id","doi"
        ])
    print(f"Initialized '{CSV_OBRA}' for writing works.")

def fetch_all_works():
    tematica_map = {}
    next_tematica_id = 1
    obra_id = 1
    page = 1
    total_results = None

    while True:
        params = {
            "page": page,
            "per_page": PER_PAGE,
            "filter": f"open_access.is_oa:true,has_content.pdf:true,primary_topic.subfield.id:{SUBFIELD_ID},best_oa_location.is_accepted:true,language:{LANGUAGE},keywords.id:{'|'.join(KEYWORDS)}",
            "sort": "cited_by_count:desc"
        }
        data = fetch_page(BASE_URL, params)
        if not data or "results" not in data:
            print(f"No data returned for page {page}, stopping.")
            break

        works = data["results"]
        if total_results is None:
            total_results = data.get("meta", {}).get("count", 0)
            print(f"Total results to fetch (approximate): {total_results}")

        print(f"Fetched page {page} with {len(works)} works.")

        with open(CSV_OBRA, "a", newline='', encoding='utf-8') as f:
            writer = csv.writer(f)
            for work in works:
                # --- PDF URL ---
                pdf_url = work.get("best_oa_location", {}).get("pdf_url")
                if not pdf_url:
                    continue

                # --- DOI ---
                doi = work.get("doi", "")

                titulo = work.get("title", "")
                abstract = reconstruct_abstract(work.get("abstract_inverted_index"))
                fecha_publicacion = work.get("publication_date", "")
                idioma = work.get("language", LANGUAGE)
                num_citas = work.get("cited_by_count", 0)
                fwci = work.get("fwci", "")
                primary_topic = work.get("primary_topic")
                if not primary_topic:
                    continue
                topic_name = primary_topic.get("display_name", "Unknown Topic")
                if topic_name not in tematica_map:
                    tematica_map[topic_name] = next_tematica_id
                    next_tematica_id += 1
                tematica_id = tematica_map[topic_name]

                writer.writerow([
                    obra_id, pdf_url, titulo, abstract, fecha_publicacion,
                    idioma, num_citas, fwci, tematica_id, doi
                ])
                obra_id += 1

        if page * PER_PAGE >= total_results or not works:
            break
        page += 1

    print(f"Finished fetching all works. Total works saved: {obra_id-1}")
    return tematica_map

def save_tematica_csv(tematica_map):
    os.makedirs(os.path.dirname(CSV_TEMATICA), exist_ok=True)
    with open(CSV_TEMATICA, "w", newline='', encoding='utf-8') as f:
        writer = csv.writer(f)
        writer.writerow(["id","nombre_campo"])
        for topic_name, topic_id in tematica_map.items():
            writer.writerow([topic_id, topic_name])
    print(f"Saved '{CSV_TEMATICA}' with {len(tematica_map)} topics.")

def update_tematica_and_generate_contenida():
    if not os.path.exists(CSV_TEMATICA):
        raise FileNotFoundError(f"{CSV_TEMATICA} not found.")

    tematicas = {}
    rows = []
    with open(CSV_TEMATICA, newline='', encoding='utf-8') as f:
        reader = csv.DictReader(f)
        for row in reader:
            row["id"] = int(row["id"])
            rows.append(row)
            tematicas[row["nombre_campo"].strip()] = row["id"]

    max_id = max(r["id"] for r in rows)
    for topic in BASE_TOPICS:
        if topic not in tematicas:
            max_id += 1
            tematicas[topic] = max_id
            rows.append({"id": max_id, "nombre_campo": topic})
            print(f"Added base topic '{topic}' with id={max_id}")

    with open(CSV_TEMATICA, "w", newline='', encoding='utf-8') as f:
        writer = csv.DictWriter(f, fieldnames=["id", "nombre_campo"])
        writer.writeheader()
        writer.writerows(rows)

    relaciones = [
        {"id_padre": tematicas["Physical Sciences"], "id_hijo": tematicas["Computer Science"]},
        {"id_padre": tematicas["Computer Science"], "id_hijo": tematicas["Artificial Intelligence"]}
    ]
    ai_id = tematicas["Artificial Intelligence"]
    for nombre, id_ in tematicas.items():
        if nombre not in BASE_TOPICS:
            relaciones.append({"id_padre": ai_id, "id_hijo": id_})
    relaciones = [{"id_padre": p, "id_hijo": h} for p, h in {(r["id_padre"], r["id_hijo"]) for r in relaciones}]

    os.makedirs(os.path.dirname(CSV_TEMATICA_CONTENIDA), exist_ok=True)
    with open(CSV_TEMATICA_CONTENIDA, "w", newline='', encoding='utf-8') as f:
        writer = csv.DictWriter(f, fieldnames=["id_padre", "id_hijo"])
        writer.writeheader()
        writer.writerows(relaciones)

    print(f"'{CSV_TEMATICA}' updated with {len(rows)} topics.")
    print(f"'{CSV_TEMATICA_CONTENIDA}' generated with {len(relaciones)} relations.")

def main():
    print("Starting OpenAlex fetch process...")
    initialize_csv_files()
    tematica_map = fetch_all_works()
    save_tematica_csv(tematica_map)
    update_tematica_and_generate_contenida()
    print("Finished fetching and processing all works and topics.")

if __name__ == "__main__":
    main()

### OBTENCIÓN DATOS NO ESTRUCTURADOS (LLM)
---
Los datos no estructurados en este sistema son las tecnologías dentro de las obras, las cuales deben extaerse manualmente desde los pdfs. Para ello, como OpenAlex tiene los _doi_ y los _url_ de las obras pero no su contenido como tal, ha sido necesario hacer _requests_ utilizando cada url de cada obra. Eso sí, según el formato de estas, la extracción del texto varía:

- Si es un PDF, conseguimos el texto procesándolo localmente.

- Si es un PDF, pero su acceso está bloqueado o no se encuentra con la URL; probamos con unpaywall y acm con el doi del artículo.

- Si no es un PDF, comprobamos que no ha devuelto un error falso (un código 200 en el que dice que no ha sido encontrado el archivo) y si nos lo ha devuelto probamos la query con unpaywall y acm con el doi del artículo.


- Si no es un PDF y no devuelve error falso iteramos por el textLabel del pdfviewer en html con _BeautifulSoup_ y scrapeamos el texto de ahí.

Con el texto ya extraido, lo siguiente es la extracción de las tecnologías. Para ello, hemos utilizado un LLM local programado dentro del código que utiliza **GPT 5.0 nano de OpenAI** para leer el código y luego extraer el texto. Ya por último se añaden las tecnologías al csv de _Tecnologias_ y se crean las relaciones dentro de *obra_tecnologia* con el id de cada obra y el nuevo id generado para cada tecnología.

In [ ]:
import csv
import os
import io
import requests
import json
import openai
import subprocess
from bs4 import BeautifulSoup
import fitz 


MODEL_NAME = "mistral:instruct"
PDF_TIMEOUT = 30
UNPAYWALL_EMAIL = "your_email@example.com"

BASE_DIR = os.path.dirname(os.path.dirname(os.path.abspath(__file__)))
CACHE_DIR = os.path.join(BASE_DIR, "cache")
OBRAS_CSV = os.path.join(CACHE_DIR, "obra.csv")
TECN_CSV = os.path.join(CACHE_DIR, "tecnologia.csv")
OBRA_TECN_CSV = os.path.join(CACHE_DIR, "obra_tecnologia.csv")


instructions = """You are a text analysis assistant specialized in identifying programming languages mentioned in academic or technical articles.
Analyze the provided raw text (extracted directly from a PDF). Identify and return the main programming languages mentioned in the article (do not include frameworks, libraries, or tools).
If a “References” or “Bibliography” section appears, ignore all text after that marker.
Return strictly in JSON, like:
{
  "programming_languages": ["Python", "C", "Java"]
}
If none found:
{
  "programming_languages": []
}
"""

os.makedirs(CACHE_DIR, exist_ok=True)

# ----------------------
# CSV helpers
# ----------------------
def read_obras_from_csv(file_path):
    obras = []
    with open(file_path, "r", newline="", encoding="utf-8") as f:
        reader = csv.DictReader(f)
        for row in reader:
            obras.append((int(row["id"]), row["direccion_fuente"], row.get("doi")))
    return obras

def init_csv(file_path, headers=None):
    os.makedirs(os.path.dirname(file_path), exist_ok=True)
    if not os.path.exists(file_path) or os.path.getsize(file_path) == 0:
        with open(file_path, "w", newline="", encoding="utf-8") as f:
            writer = csv.writer(f)
            if headers:
                writer.writerow(headers)
        return 1
    max_id = 0
    with open(file_path, "r", newline="", encoding="utf-8") as f:
        try:
            reader = csv.DictReader(f)
            for row in reader:
                val = row.get("id")
                if val:
                    try:
                        max_id = max(max_id, int(val))
                    except:
                        continue
        except:
            f.seek(0)
            for line in f:
                parts = line.split(",")
                if parts:
                    try:
                        max_id = max(max_id, int(parts[0].strip()))
                    except:
                        continue
    return max_id + 1

def append_unique_to_csv(file_path, row, headers=None, key_index=1):
    init_csv(file_path, headers=headers)
    existing_keys = set()
    with open(file_path, "r", newline="", encoding="utf-8") as f:
        reader = csv.reader(f)
        peek = next(reader, None)
        if headers and peek and all(h in peek for h in headers):
            pass
        else:
            if peek:
                try:
                    existing_keys.add(peek[key_index])
                except:
                    pass
        for r in reader:
            try:
                existing_keys.add(r[key_index])
            except:
                continue
    key = row[key_index] if len(row) > key_index else None
    if key not in existing_keys:
        with open(file_path, "a", newline="", encoding="utf-8") as f:
            writer = csv.writer(f)
            writer.writerow(row)



def append_to_csv(file_path, row, headers=None):
    """Simple append without uniqueness (for obra_tecnologia)"""
    file_exists = os.path.exists(file_path)
    with open(file_path, "a", newline="", encoding="utf-8") as f:
        writer = csv.writer(f)
        if not file_exists and headers:
            writer.writerow(headers)
        writer.writerow(row)
def load_tecnologias(file_path):
    tech_map = {}
    if os.path.exists(file_path):
        with open(file_path, newline="", encoding="utf-8") as f:
            reader = csv.DictReader(f)
            for row in reader:
                tech_map[row["nombre"]] = int(row["id"])
    return tech_map

# ----------------------
# PDF + Analysis
# ----------------------
def get_text_from_pdf_url(pdf_url, doi=None):
    tried_urls = set()
    headers = {'User-Agent': 'Mozilla/5.0'}
    
    def fetch_unpaywall_pdf(doi):
        if not doi:
            return None
        try:
            unpaywall_url = f"https://api.unpaywall.org/v2/{doi}?email={UNPAYWALL_EMAIL}"
            r = requests.get(unpaywall_url, timeout=10)
            if r.status_code == 200:
                data = r.json()
                pdf_link = data.get("best_oa_location", {}).get("url_for_pdf")
                if pdf_link:
                    print(f"📖 Found Unpaywall PDF: {pdf_link}")
                    return pdf_link
        except Exception as e:
            print(f"⚠️ Unpaywall fetch failed: {e}")
        return None


def get_text_from_pdf_url(pdf_url, doi=None):
    tried_urls = set()
    headers = {'User-Agent': 'Mozilla/5.0'}
    
    def fetch_unpaywall_pdf(doi):
        if not doi:
            return None
        try:
            unpaywall_url = f"https://api.unpaywall.org/v2/{doi}?email={UNPAYWALL_EMAIL}"
            r = requests.get(unpaywall_url, timeout=10)
            if r.status_code == 200:
                data = r.json()
                pdf_link = data.get("best_oa_location", {}).get("url_for_pdf")
                if pdf_link:
                    print(f"📖 Found Unpaywall PDF: {pdf_link}")
                    return pdf_link
        except Exception as e:
            print(f"⚠️ Unpaywall fetch failed: {e}")
        return None

    def fetch_acm_pdf(doi):
        if not doi:
            return None
        try:
            acm_url = f"https://dl.acm.org/doi/pdf/{doi}"
            r = requests.head(acm_url, allow_redirects=True, timeout=10)
            if r.status_code == 200 and "pdf" in r.headers.get("Content-Type", "").lower():
                print(f"📄 Found ACM PDF: {acm_url}")
                return acm_url
        except Exception as e:
            print(f"⚠️ ACM fetch failed: {e}")
        return None

    while pdf_url and pdf_url not in tried_urls:
        tried_urls.add(pdf_url)
        try:
            response = requests.get(pdf_url, headers=headers, timeout=PDF_TIMEOUT)
            content_type = response.headers.get("Content-Type", "").lower()

            if "application/pdf" in content_type:
                try:
                    pdf_bytes = io.BytesIO(response.content)
                    doc = fitz.open(stream=pdf_bytes, filetype="pdf")
                    text = "\n\n".join([page.get_text() for page in doc])
                    if not text.strip():
                        raise ValueError("No text extracted from PDF")
                    return text.strip(), pdf_url
                except Exception as e:
                    print(f"⚠️ PDF parse error with PyMuPDF: {e}")
                pdf_url = fetch_unpaywall_pdf(doi) or fetch_acm_pdf(doi)
                continue

            if "text/html" in content_type:
                soup = BeautifulSoup(response.text, "html.parser")
                body_text = soup.get_text(separator=' ', strip=True).lower()
                if any(x in body_text for x in ["not found", "error 404", "no encontrado", "access denied"]):
                    pdf_url = fetch_unpaywall_pdf(doi) or fetch_acm_pdf(doi)
                    continue
                text_labels = soup.select('[class*="textLayer"], [id*="textLayer"] div, span')
                texts = [el.get_text(separator=' ', strip=True) for el in text_labels]
                if texts:
                    return " ".join(texts), pdf_url
                pdf_links = [a['href'] for a in soup.find_all('a', href=True) if a['href'].endswith('.pdf')]
                if pdf_links:
                    next_pdf = requests.compat.urljoin(pdf_url, pdf_links[0])
                    if next_pdf not in tried_urls:
                        pdf_url = next_pdf
                        continue
                pdf_url = fetch_unpaywall_pdf(doi) or fetch_acm_pdf(doi)
                continue

            pdf_url = fetch_unpaywall_pdf(doi) or fetch_acm_pdf(doi)

        except Exception as e:
            print(f"⚠️ Exception while fetching PDF: {e}")
            pdf_url = fetch_unpaywall_pdf(doi) or fetch_acm_pdf(doi)

    print("❌ No valid PDF or text found.")
    return None, None



def estimate_tokens(text: str) -> int:
    """Estimate the number of tokens in a string for GPT models."""
    return len(ENCODING.encode(text))

def analyze_text(instructions, pdf_text):
    detected_languages = set()

    # 1️⃣ Try LLM first
    if pdf_text.strip():
        prompt = f"{instructions}\n\nText:\n{pdf_text}"
        try:
            result = subprocess.run(
                ["ollama", "run", MODEL_NAME],
                input=prompt,
                capture_output=True,
                text=True,
                encoding="utf-8",
                errors="ignore"
            )
            raw = result.stdout.strip()
            json_start = raw.find("{")
            json_end = raw.rfind("}") + 1
            if json_start != -1 and json_end != -1:
                llm_result = json.loads(raw[json_start:json_end])
                detected_languages.update(llm_result.get("programming_languages", []))
        except Exception:
            pass

    return {"programming_languages": sorted(detected_languages)}

# Make sure to set your API key in the environment
# export OPENAI_API_KEY="sk-..."
def analyze_text_with_gpt(pdf_text, model="gpt-5-nano"):
    """
    Analyze PDF text using ChatGPT Responses API.
    Returns a set of detected programming languages.
    Automatically skips blocked content.
    """
    detected_languages = set()

    # Blocked content check
    blocked_indicators = [
        "enable javascript and cookies to continue",
        "access denied",
        "not found",
        "error 404"
    ]
    preview_text = pdf_text[:300].replace("\n", " ").lower()
    if any(b in preview_text for b in blocked_indicators):
        print("⚠️ Blocked content detected, skipping analysis.")
        return {"programming_languages": []}

    # GPT-5 request
    try:
        response = openai.responses.create(
            model=model,
            input=f"""
            You are a text analysis assistant specialized in identifying programming languages mentioned in academic or technical articles.

            Task:
            1️⃣ Identify **only actual programming languages** used to write code.
            2️⃣ Do **NOT** include frameworks, libraries, standards, formal languages, or platforms.
            3️⃣ Ignore any text after a “References” or “Bibliography” section.
            4️⃣ Return strictly in JSON:

            {{"programming_languages": ["Python", "C", "Java"]}}

            If none found, return:

            {{"programming_languages": []}}


            Text:
            {pdf_text}
"""
        )
        raw = response.output_text.strip()
        json_start = raw.find("{")
        json_end = raw.rfind("}") + 1
        if json_start != -1 and json_end != -1:
            llm_result = json.loads(raw[json_start:json_end])
            detected_languages.update(llm_result.get("programming_languages", []))
    except Exception as e:
        print(f"⚠️ GPT analysis failed: {e}")

    return {"programming_languages": sorted(detected_languages)}

# ----------------------
# Main loop
# ----------------------
def process_all_obras():
    obras = read_obras_from_csv(OBRAS_CSV)
    print(f"Found {len(obras)} obras in CSV.")

    next_tecn_id = init_csv(TECN_CSV, headers=["id","nombre"])
    next_link_id = init_csv(OBRA_TECN_CSV, headers=["id","obra_id","tecnologia_id"])

    for obra_id, pdf_url, doi in obras:
        print(f"\n🔹 Processing Obra ID: {obra_id}")
        try:
            # Step 1: fetch PDF text
            text, final_url = get_text_from_pdf_url(pdf_url, doi)
            if not text:
                print(f"❌ No valid PDF or text found.")
                print(f"⚠️ Skipping Obra ID {obra_id}, no text extracted.")
                continue
            else:
                preview = text[:300].replace("\n", " ").strip()
                print(f"📄 Text extracted for Obra ID {obra_id} ({len(text)} chars)")
                print(f"🔗 Source URL used: {final_url}")
                print(f"📝 Text preview: {preview}{'...' if len(text) > 300 else ''}")

            # Step 2: analyze with Ollama
            print(f"🤖 Analyzing text for Obra ID {obra_id}...")
            try:
                result = analyze_text_with_gpt(text)
            except Exception as e:
                print(f"⚠️ Analysis failed for Obra ID {obra_id}: {e}")
                result = {"programming_languages": []}

            languages = result.get("programming_languages", [])
            print(f"📝 Obra ID {obra_id} languages detected: {languages}")

            tech_map = load_tecnologias(TECN_CSV)  # { "Python": 89, "C": 90, ... }

            for lang in languages:
                if lang not in tech_map:
                    tech_map[lang] = next_tecn_id
                    append_to_csv(TECN_CSV, [next_tecn_id, lang], headers=["id","nombre"])
                    next_tecn_id += 1

                tecnologia_id = tech_map[lang]
                append_to_csv(OBRA_TECN_CSV, [next_link_id, obra_id, tecnologia_id], headers=["id","obra_id","tecnologia_id"])
                next_link_id += 1

        except Exception as e:
            print(f"⚠️ Unexpected error processing Obra ID {obra_id}: {e}")


if __name__ == "__main__":
    #Uncomment if you are testing runs 
    """for csv_file in [TECN_CSV, OBRA_TECN_CSV]:
        if os.path.exists(csv_file):
            os.remove(csv_file)
            print(f"🗑️ Deleted old CSV: {csv_file}")"""
    process_all_obras()

### ALMACENAMIENTO EN POSTGRESQL
---

Con los csvs llenos y PostgreSQL (por docker compose) conectado y con la BD creada, lo siguiente es llenarla. Como en el algoritmo de creación de la BD, es necesario pasar los parámetros de la base de datos para realizar la conexión con PostgreSQL: **host**, **port**, **database**, **user** y **password**. Con esta, ya tenemos el cursosr y podemos empezar con el proceso.

La idea aquí es extraer los datos de los csvs con **Dataframes de pandas** y con estos hacer las distintas consultas para insertar (_INSERT into ..._) los dataframes en la BD. Por último, se realiza el commit y se cierra el cursosr y la conexión.

In [ ]:
import os
import pandas as pd
import psycopg2

DB_PARAMS = {
    "host": "localhost",
    "port": 5432,
    "database": "demoDB",
    "user": "userPSQL",
    "password": "passPSQL"
}

def main():
    connection = psycopg2.connect(**DB_PARAMS)
    cursor = connection.cursor()

    script_dir = os.path.dirname(os.path.abspath(__file__))
    dir_cache = os.path.join(script_dir, '../cache')

    # File paths
    file_tematica = os.path.join(dir_cache, 'tematica.csv')
    file_tematica_contenida = os.path.join(dir_cache, 'tematica_contenida.csv')
    file_obra = os.path.join(dir_cache, 'obra.csv')
    file_tecnologia = os.path.join(dir_cache, 'tecnologia.csv')
    file_obra_tecnologia = os.path.join(dir_cache, 'obra_tecnologia.csv')

    # Read CSVs
    df_tematica = pd.read_csv(file_tematica)
    df_tematica_contenida = pd.read_csv(file_tematica_contenida)
    df_obra = pd.read_csv(file_obra)
    df_tecnologia = pd.read_csv(file_tecnologia)
    df_obra_tecnologia = pd.read_csv(file_obra_tecnologia)

    # Insert tematica
    for _, row in df_tematica.iterrows():
        cursor.execute("""
            INSERT INTO tematica (id, nombre_campo)
            VALUES (%s, %s)
            ON CONFLICT (id) DO NOTHING
        """, (int(row['id']), row['nombre_campo'].strip() if pd.notna(row['nombre_campo']) else None))

    # Insert tematica_contenida
    for _, row in df_tematica_contenida.iterrows():
        cursor.execute("""
            INSERT INTO tematica_contenida (id, tematica_padre_id, tematica_hijo_id)
            VALUES (%s, %s, %s)
            ON CONFLICT DO NOTHING
        """, (int(row['id']), int(row['id_padre']), int(row['id_hijo'])))

    # Insert obra
    for _, row in df_obra.iterrows():
        cursor.execute("""
            INSERT INTO obra (
                id, doi, direccion_fuente, titulo, abstract, fecha_publicacion,
                idioma, num_citas, fwci, tematica_id
            ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
            ON CONFLICT (id) DO NOTHING
        """, (
            int(row['id']),
            row.get('doi').strip() if pd.notna(row.get('doi')) else None,
            row['direccion_fuente'].strip() if pd.notna(row.get('direccion_fuente')) else None,
            row['titulo'].strip() if pd.notna(row.get('titulo')) else None,
            row.get('abstract').strip() if pd.notna(row.get('abstract')) else None,
            row.get('fecha_publicacion') if pd.notna(row.get('fecha_publicacion')) else None,
            row.get('idioma').strip() if pd.notna(row.get('idioma')) else None,
            int(row.get('num_citas', 0)) if pd.notna(row.get('num_citas')) else 0,
            float(row.get('fwci', 0.0)) if pd.notna(row.get('fwci')) else 0.0,
            int(row.get('tematica_id')) if pd.notna(row.get('tematica_id')) else None
        ))

    # Insert tecnologia
    for _, row in df_tecnologia.iterrows():
        cursor.execute("""
            INSERT INTO tecnologia (id, nombre, tipo, version)
            VALUES (%s, %s, %s, %s)
            ON CONFLICT (id) DO NOTHING
        """, (
            int(row['id']),
            row['nombre'].strip() if pd.notna(row['nombre']) else None,
            row.get('tipo').strip() if pd.notna(row.get('tipo')) else None,
            row.get('version').strip() if pd.notna(row.get('version')) else None
        ))

    # Insert obra_tecnologia
    for _, row in df_obra_tecnologia.iterrows():
        cursor.execute("""
            INSERT INTO obra_tecnologia (id, obra_id, tecnologia_id)
            VALUES (%s, %s, %s)
            ON CONFLICT (id) DO NOTHING
        """, (int(row['id']), int(row['obra_id']), int(row['tecnologia_id'])))
# Puede que on conflict falle por lo de id 
    connection.commit()
    cursor.close()
    connection.close()
    print("✅ CSV data loaded successfully including tecnologia and obra_tecnologia.")

if __name__ == "__main__":
    main()


### CREACIÓN DEL GRAFO (TURTLE, SCHEMA & SKOS)
---
Para la importación de datos a GraphDB hemos seguido la lógica de utilizar  _Schema_ y _SKOS_ como indicamos en el apartado inicial. A esto se le suma la librería _Graph_ de _Python_, que es la encargada de crear el grafo del sistema. Para la conexión de la BD, se sigue utilizando la lógica de conexión con cursor. Como la base de datos está conectada, se utilizan los datos directamente ahí para crear el archivo _Turtle_.

Para cada tabla, se selecciona su conjunto de datos completo y a partir de aquí el proceso cmabia si es una entidad o una relación:

* **Entidad:** Aquí primero se identifica a cada objeto con un identificador con la estrutura *tabla_uri*. Este es el nombre del nodo correspondiente al objeto. Posteriormente, con SKOS o Schema (según la tabla), se añade el resto de datos al nodo.
* **Relación:** Aquí la etiqueta *tabla_uri* solamente es necesaria para *tematica_contenida* ya que es una relación entre una tabla consigo misma. Para representar esta relación fue necesario el uso de _SKOS_ ya que sus funciones _narrower_ y _broader_ representan la relación padre-hijo e hijo-padre respectivamente. Para *obra_tecnologia* por ser entre distintas tablas, solo hacía falta reprsentar la relación con _Schema_.

Con el grafo terminado, se serializa y se almacena en la carpeta **ttl** como *openalex_graph.ttl*. Con este archivo solo haría falta subirlo al servidor de GraphDB y ya allí hacer las consultas en _SPARQL_.

In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

import psycopg2
from rdflib import Graph, Namespace, URIRef, Literal
from rdflib.namespace import RDF, SKOS, XSD

# --- Namespaces ---
SCHEMA = Namespace("https://schema.org/")
OPENALEX = Namespace("https://openalex.org/")
g = Graph()
g.bind("schema", SCHEMA)
g.bind("skos", SKOS)
g.bind("openalex", OPENALEX)

# --- PostgreSQL connection ---
DB_PARAMS = {
    "host": "localhost",
    "port": 5432,
    "database": "demoDB",
    "user": "userPSQL",
    "password": "passPSQL"
}


conn = psycopg2.connect(**DB_PARAMS)

cur = conn.cursor()

print("Connected to database ✅")

# --- 1. TEMÁTICA ---
cur.execute("SELECT id, nombre_campo FROM tematica;")
for tmid, nombre in cur.fetchall():
    tema_uri = OPENALEX[f"tematica_{tmid}"]
    g.add((tema_uri, RDF.type, SKOS.Concept))
    g.add((tema_uri, SKOS.prefLabel, Literal(nombre)))

print("Mapped table: tematica ✅")

# tematica_contenida → skos:broader
cur.execute("SELECT id, tematica_padre_id, tematica_hijo_id FROM tematica_contenida;")
for tcid, parent, child in cur.fetchall():
    parent_uri = OPENALEX[f"tematica_{parent}"]
    child_uri = OPENALEX[f"tematica_{child}"]
    g.add((parent_uri, SKOS.narrower, child_uri))
    g.add((child_uri, SKOS.broader, parent_uri))

# --- 2. TECNOLOGÍA ---
cur.execute("SELECT id, nombre, tipo, version FROM tecnologia;")
for tid, nombre, tipo, version in cur.fetchall():
    tech_uri = OPENALEX[f"tecnologia_{tid}"]
    g.add((tech_uri, RDF.type, SCHEMA.SoftwareApplication))
    g.add((tech_uri, SCHEMA.name, Literal(nombre)))
    if tipo:
        g.add((tech_uri, SCHEMA.applicationCategory, Literal(tipo)))
    if version:
        g.add((tech_uri, SCHEMA.softwareVersion, Literal(version)))

print("Mapped table: tecnologia ✅")

# --- 3. OBRA ---
cur.execute("SELECT id, doi, direccion_fuente, titulo, abstract, fecha_publicacion, idioma, num_citas, fwci, tematica_id FROM obra;")
for oid, doi, direccion_fuente, titulo, abstract, fecha_publicacion, idioma, num_citas, fwci, tematica_id in cur.fetchall():
    obra_uri = OPENALEX[f"obra_{oid}"]
    g.add((obra_uri, RDF.type, SCHEMA.TechArticle))
    if doi:
        g.add((obra_uri, SCHEMA.sameAs, Literal(doi)))
    if direccion_fuente:
        g.add((obra_uri, SCHEMA.url, Literal(direccion_fuente)))
    if titulo:
        g.add((obra_uri, SCHEMA.name, Literal(titulo)))
    if abstract:
        g.add((obra_uri, SCHEMA.abstract, Literal(abstract)))
    if fecha_publicacion:
        g.add((obra_uri, SCHEMA.datePublished, Literal(fecha_publicacion, datatype=XSD.date)))
    if idioma:
        g.add((obra_uri, SCHEMA.inLanguage, Literal(idioma)))
    if num_citas:
        g.add((obra_uri, SCHEMA.citationCount, Literal(num_citas, datatype=XSD.integer)))
    if fwci:
        g.add((obra_uri, SCHEMA.metric, Literal(fwci, datatype=XSD.float)))
    if tematica_id:
        g.add((obra_uri, SCHEMA.about, OPENALEX[f"tematica_{tematica_id}"]))

print("Mapped table: obra ✅")

# obra_tecnologia → schema:mentions
cur.execute("SELECT obra_id, tecnologia_id FROM obra_tecnologia;")
for oid, tid in cur.fetchall():
    g.add((OPENALEX[f"obra_{oid}"], SCHEMA.mentions, OPENALEX[f"tecnologia_{tid}"]))


print("Mapped relationships ✅")

# --- 4. EXPORT ---
output_file = "ttl/openalex_graph.ttl"
g.serialize(destination=output_file, format="turtle")
print(f"RDF graph exported to {output_file} 🧩")

# --- 5. Cleanup ---
cur.close()
conn.close()
print("PostgreSQL connection closed 🔒")


### CREACIÓN DE CONSULTAS
---

Para finalizar el proyecto, hemos creado una serie de querys con las que se pueda analizar sí el sistema cumple los objetivos programados.

Primeramente, se probó si cumplía el objetivo principal: obtener el número de apariciones de distintos lenguajes de programación en obras de cada subtemática (o temática sin hijas) distinta. El resultado fue una tabla donde se listaban todas las combinaciones de tópico y lenguaje distinto dentro de la base de datos, junto al número de veces que se repetía esa relación. Por lo tanto, se cumplió la meta descrita al inicio del proyecto.

Para demostrar su funcionamiento, solo hay que utilizar la siguiente consulta en GraphDB:

In [ ]:
"""
PREFIX schema: <https://schema.org/>
PREFIX skos: <http://www.w3.org/2004/02/skos/core#>

# Queremos una lista de "aristas" (conexiones)
SELECT ?topicName ?techName (COUNT(DISTINCT ?work) AS ?sharedWorksCount)
WHERE {
    ?work schema:about ?topic .
    ?work schema:mentions ?technology .
    
    OPTIONAL { ?topic skos:prefLabel ?topicName . }
    OPTIONAL { ?technology schema:name ?techName . }
    
    FILTER(BOUND(?topicName) && BOUND(?techName))
}
GROUP BY ?topicName ?techName
ORDER BY DESC(?sharedWorksCount)

"""

Posteriormente, para exprimir los límites del sistema, decidimos probar una consulta más: Tecnologías más competitivas entre campos. En esta consulta, se lista el número de veces que cada par de tecnologías distintas (por ejemplo, Python y Java, C++ y Prolog, etc.) aparecen en las mismas obras. Con el resultado, aprendimos que Python y Java eran la dupla que más aparecía en las mismas obras.

Para demostrar su funcionamiento, solo hay que utilizar la siguiente consulta en GraphDB:

In [ ]:
"""
PREFIX schema: <https://schema.org/>

SELECT ?techA_name ?techB_name (COUNT(DISTINCT ?topic) AS ?commonTopics)
WHERE {
    # Encuentra la primera tecnología (A) mencionada por una obra
    ?workA schema:mentions ?techA .
    ?workA schema:about ?topic .
    ?techA schema:name ?techA_name .

    # Encuentra la segunda tecnología (B) mencionada por la misma obra (o una obra sobre el mismo tema)
    ?workB schema:mentions ?techB .
    ?workB schema:about ?topic . # <-- Mismo tema
    ?techB schema:name ?techB_name .

    # Asegura que no sea la misma tecnología
    FILTER (?techA != ?techB)

    # Solo queremos los nombres, no las URIs largas
    FILTER(BOUND(?techA_name) && BOUND(?techB_name))
}
GROUP BY ?techA_name ?techB_name
ORDER BY DESC(?commonTopics)
LIMIT 10

"""